### Prelude:

In [27]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
set_seed = 42

In [8]:
data = pd.read_csv("Datasets/cleaned_data.csv")

In [9]:
data.head()

,GasTax,CarbTax,Treaty,RegCarb,political_views,party_id,party_id.1,party_id.2,QID74,ScientificConfidence,...,GasTax_after,CarbTax_after,Treaty_after,RegCarb_after,treatment_value,party_id_merged,mean_climate_support_before,mean_climate_support_after,mean_climate_support_change,treatment_frame
0,1.0,1.0,2.0,2.0,3.0,Independent,0,0,3,4.0,...,1.0,1.0,2.0,2.0,6,3,1.50,1.50,0.00,Efficiency
1,2.0,2.0,3.0,3.0,1.0,Independent,0,0,2,5.0,...,2.0,2.0,3.0,3.0,0,2,2.50,2.50,0.00,No framing
2,2.0,3.0,3.0,3.0,0.0,Democrat,0,0,0,4.0,...,3.0,3.0,3.0,3.0,4,0,2.75,3.00,0.25,Secular
3,1.0,1.0,1.0,2.0,4.0,Independent,0,0,4,4.0,...,1.0,1.0,1.0,2.0,4,4,1.25,1.25,0.00,Secular
4,1.0,3.0,3.0,3.0,1.0,Democrat,0,1,0,5.0,...,2.0,3.0,3.0,3.0,1,1,2.50,2.75,0.25,Positive science


Adapted from Tiffanie's Code:

In [10]:
#from Tiffanie's code
# distribution of subjects across treatment conditions (like Table 1 from paper)
# N = 186
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.118280
1,Positive science,22,0.118280
2,Negative science,18,0.096774
3,Religious,35,0.188172
4,Secular,30,0.161290
5,Equity,32,0.172043
6,Efficiency,27,0.145161


In [11]:
#Adapted from Tiffanie's code
pd.pivot_table(data, values=['mean_climate_support_before', 'mean_climate_support_after', 'mean_climate_support_change'],
               index=['treatment_value','treatment_frame'],
               aggfunc=['mean'])

mean  \
                                 mean_climate_support_after   
treatment_value treatment_frame                               
0               No framing                         2.012500   
1               Positive science                   2.125000   
2               Negative science                   2.416667   
3               Religious                          2.038095   
4               Secular                            2.080556   
5               Equity                             2.213889   
6               Efficiency                         2.126543   

                                                              \
                                 mean_climate_support_before   
treatment_value treatment_frame                                
0               No framing                          2.016667   
1               Positive science                    1.975000   
2               Negative science                    2.283333   
3               Religious                           1.997619   
4               Secular                             2.022222   
5               Equity                              2.134409   
6               Efficiency                          2.126543   

                                                              
                                 mean_climate_support_change  
treatment_value treatment_frame                               
0               No framing                     -4.166667e-03  
1               Positive science                1.500000e-01  
2               Negative science                1.333333e-01  
3               Religious                       4.047619e-02  
4               Secular                         5.833333e-02  
5               Equity                          7.500000e-02  
6               Efficiency                     -2.055969e-18

In [12]:
pd.pivot_table(data, values=["mean_climate_support_after"],
               index=["party_id_merged", "treatment_value", "treatment_frame"], aggfunc=['mean', 'sem'])

mean  \
                                                 mean_climate_support_after   
party_id_merged treatment_value treatment_frame                               
0               0               No framing                         2.416667   
                1               Positive science                   2.250000   
                2               Negative science                   2.656250   
                3               Religious                          2.281250   
                4               Secular                            2.312500   
                5               Equity                             2.694444   
                6               Efficiency                         2.093750   
1               0               No framing                         1.809524   
                1               Positive science                   2.500000   
                2               Negative science                   2.500000   
                3               Religious                          2.022727   
                4               Secular                            1.888889   
                5               Equity                             2.083333   
                6               Efficiency                         2.000000   
2               0               No framing                         2.250000   
                1               Positive science                   2.166667   
                2               Negative science                   2.166667   
                3               Religious                          2.083333   
                4               Secular                            2.416667   
                5               Equity                             2.208333   
                6               Efficiency                         2.500000   
3               0               No framing                         2.000000   
                1               Positive science                   1.416667   
                2               Negative science                   1.750000   
                3               Religious                          1.875000   
                4               Secular                            2.104167   
                5               Equity                             1.968750   
                6               Efficiency                         2.062500   
4               0               No framing                         1.666667   
                1               Positive science                   2.000000   
                4               Secular                            1.250000   
                5               Equity                             2.000000   
                6               Efficiency                         2.000000   
5               1               Positive science                   2.875000   
                2               Negative science                   1.750000   
                3               Religious                          1.694444   
                4               Secular                            2.000000   
                5               Equity                             2.250000   
                6               Efficiency                         1.875000   
6               1               Positive science                   1.250000   
                3               Religious                          2.000000   
                4               Secular                            1.666667   
                6               Efficiency                         1.666667   

                                                                        sem  
                                                 mean_climate_support_after  
party_id_merged treatment_value treatment_frame                              
0               0               No framing                         0.463980  
                1               Positive science                   0.079057  
                2               Negative scien

In [25]:
def demean(dataframe, variable):
    data[f'{variable}_demeaned'] = data[variable] - data[variable].mean()

In [26]:
#Demean religiosity and scientific confidence for covariate analysis
covariates = ['Religiosity', 'Economic_Reasoning', 'ScientificConfidence']
for cov in covariates: demean(data, cov)

In [21]:
#Shuffle the dataset and split into two random folds. Set_seed set at 42.
shuffled = data.sample(random_state=set_seed, frac=1)
split = np.array_split(shuffled, 2)
fold1 = split[0]
fold2 = split[1]

/opt/homebrew/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [22]:
pd.pivot_table(fold1, values=["mean_climate_support_after"],
               index=["treatment_value", "treatment_frame"], aggfunc=['mean', 'sem'])

,,mean,sem
,,mean_climate_support_after,mean_climate_support_after
treatment_value,treatment_frame,,
0,No framing,2.062500,0.257694
1,Positive science,2.218750,0.173189
2,Negative science,2.250000,0.186339
3,Religious,2.105263,0.132122
4,Secular,2.055556,0.130035
5,Equity,2.263158,0.100208
6,Efficiency,2.266667,0.179505


In [23]:
pd.pivot_table(fold2, values=["mean_climate_support_after"],
               index=["treatment_value", "treatment_frame"], aggfunc=['mean', 'sem'])

,,mean,sem
,,mean_climate_support_after,mean_climate_support_after
treatment_value,treatment_frame,,
0,No framing,1.979167,0.160598
1,Positive science,2.062500,0.160270
2,Negative science,2.666667,0.139443
3,Religious,1.958333,0.142725
4,Secular,2.105556,0.132753
5,Equity,2.128788,0.209426
6,Efficiency,2.044118,0.147680


In [ ]:
def lin_estimator_mult_treat(data, y_var, treatment_vars, covariate_list):
    # Demean the covariates
    for cov in covariate_list:
        data[cov + '_demeaned'] = data[cov] - data[cov].mean()

    # Create interaction terms for each treatment and each demeaned covariate
    for treat in treatment_vars:
        for cov in covariate_list:
            data[treat + '_X_' + cov] = data[treat] * data[cov + '_demeaned']

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_vars)
    
    # Include each demeaned covariate
    covariates_formula = " + ".join([cov + '_demeaned' for cov in covariate_list])
    
    # Include each interaction term
    interactions_formula = " + ".join([treat + '_X_' + cov for treat in treatment_vars for cov in covariate_list])

    # Full formula
    formula = f"{y_var} ~ {treatments_formula} + {covariates_formula} + {interactions_formula}"

    # Fit the regression model
    model = sm.OLS.from_formula(formula, data=data).fit()

    return model.summary()
